## Feature Construction

In [ ]:
# Import necessary libraries
import os
from sklearn.impute import KNNImputer
from general_used_functions import *
from collections import defaultdict
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [ ]:
# Load config data
config_data = load_config_file()

# Load the data from the data directory
DATA_DIR = os.getcwd() + '/data'
training_stock_df = defaultdict(list)
testing_stock_df = defaultdict(list)
stock_list = config_data['stock_dict'].keys()

# Load traininig data
for stock in stock_list:
    training_stock_data = pd.read_excel(f"{DATA_DIR}/stock_price_data/training/{stock}_stock_price_data(training).xlsx")
    training_stock_df[stock] = training_stock_data

# Load testing data
for stock in stock_list:
    testing_stock_data = pd.read_excel(f"{DATA_DIR}/stock_price_data/testing/{stock}_stock_price_data(testing).xlsx")
    testing_stock_df[stock] = testing_stock_data

In [ ]:
# Initialize feature dataframe and store the date as the first column
training_feature_df = {}
testing_feature_df = {}

# Training
for stock in stock_list:
    training_feature_df[stock] = pd.DataFrame()
    training_feature_df[stock]['date'] = training_stock_df[stock]['date']

# Testing
for stock in stock_list:
    testing_feature_df[stock] = pd.DataFrame()
    testing_feature_df[stock]['date'] = testing_stock_df[stock]['date']

#### Feature 1 - 3: Exponential Weighted Moving (EMW) with halflives(days) = 5, 10 and 21  (Self-Constructed)

In [ ]:
def exponential_moving_average(data, halflives):
    return data.ewm(halflife=halflives).mean()

# Training
for stock in stock_list:
    for halflife in [5, 10, 21]:
        training_feature_df[stock][f'{stock}_exponential_moving_average_{halflife}'] = exponential_moving_average(training_stock_df[stock][stock], halflife)

        if check_weird_data(training_feature_df[stock][f'{stock}_exponential_moving_average_{halflife}']):
            print(f"Feature {stock}_exponential_moving_average_{halflife} contains weird data in training data")

# Testing
for stock in stock_list:
    for halflife in [5, 10, 21]:
        testing_feature_df[stock][f'{stock}_exponential_moving_average_{halflife}'] = exponential_moving_average(testing_stock_df[stock][stock], halflife)

        if check_weird_data(testing_feature_df[stock][f'{stock}_exponential_moving_average_{halflife}']):
            print(f"Feature {stock}_exponential_moving_average_{halflife} contains weird data in testing data")

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 4 - 6: EWM downside deviation (DD) in log scale with halflives(days) = 5, 10, 21 (Self-Constructed)

In [ ]:
def weighted_moving_downside_deviation_with_log(data, halflives, TRADING_DAYS=252):
    log_returns = np.log(data / data.shift(1))
    downside_returns = log_returns[log_returns < 0].fillna(0)
    ewm_std = downside_returns.ewm(halflife=halflives).std()
    annualized_ewm_std = ewm_std * np.sqrt(TRADING_DAYS)
    return annualized_ewm_std

# Initialize a dictionary to store weird columns for each stock
training_weird_columns_dict = {stock: [] for stock in stock_list}
testing_weird_columns_dict = {stock: [] for stock in stock_list}

# Training
for stock in stock_list:
    for halflife in [5, 10, 21]:
        downside_deviation = weighted_moving_downside_deviation_with_log(training_stock_df[stock][stock], halflife)
        downside_deviation = downside_deviation.reindex(training_stock_df[stock].index) 
        training_feature_df[stock][f'{stock}_downside_deviation_{halflife}'] = downside_deviation

        if check_weird_data(training_feature_df[stock][f'{stock}_downside_deviation_{halflife}']):
            print(f"Feature {stock}_downside_deviation_{halflife} contains weird data")
            training_weird_columns_dict[stock].append(f'{stock}_downside_deviation_{halflife}')

# Testing
for stock in stock_list:
    for halflife in [5, 10, 21]:
        downside_deviation = weighted_moving_downside_deviation_with_log(testing_stock_df[stock][stock], halflife)
        downside_deviation = downside_deviation.reindex(testing_stock_df[stock].index) 
        testing_feature_df[stock][f'{stock}_downside_deviation_{halflife}'] = downside_deviation

        if check_weird_data(testing_feature_df[stock][f'{stock}_downside_deviation_{halflife}']):
            print(f"Feature {stock}_downside_deviation_{halflife} contains weird data")
            testing_weird_columns_dict[stock].append(f'{stock}_downside_deviation_{halflife}')

In [ ]:
# handle the weird data with KNN imputer
def handle_weird_data_with_knn_imputer(data, weird_columns):
    imputer = KNNImputer(n_neighbors=5)
    data[weird_columns] = imputer.fit_transform(data[weird_columns])
    return data


# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

In [ ]:
# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")

In [ ]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

for stock in stock_list:
    testing_weird_columns_dict[stock] = []

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 7 - 9: Sortino ratio (Self-Constructed)

In [ ]:
# First, calculate the daily return of the stock
def daily_return(stock_price):
    return stock_price.pct_change()

# Training
for stock, stock_data in training_stock_df.items():
    stock_data[f'{stock}_daily_return'] = daily_return(stock_data[stock])
    if check_weird_data(stock_data[f'{stock}_daily_return']):
            print(f"{stock} daily return contains weird data in training data")
            training_weird_columns_dict[stock].append(f'{stock}_daily_return')

# Testing
for stock, stock_data in testing_stock_df.items():
    stock_data[f'{stock}_daily_return'] = daily_return(stock_data[stock])
    if check_weird_data(stock_data[f'{stock}_daily_return']):
            print(f"{stock} daily return contains weird data in testing data")
            testing_weird_columns_dict[stock].append(f'{stock}_daily_return')

In [ ]:
# Handle the weird data with KNN imputer
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_stock_df[stock] = handle_weird_data_with_knn_imputer(training_stock_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_stock_df[stock] = handle_weird_data_with_knn_imputer(testing_stock_df[stock], weird_columns)

In [ ]:
# Check the data again
# Training
for stock, stock_data in training_stock_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

# Testing
for stock, stock_data in testing_stock_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")

In [ ]:
# Clear weird_columns_dict
for stock in stock_list: 
    training_weird_columns_dict[stock] = []
    testing_weird_columns_dict[stock] = []

In [ ]:
# Before calculating the Sortino ratio, 
# we need to append the daily risk free rate and 
# downside deviation to the stock data

daily_risk_free_rate = pd.read_excel(f"{DATA_DIR}/risk_free_rate/(Fama-French 3 Factors Plus Momentum - Daily Frequency)Risk-Free_Return_Rate.xlsx")
daily_risk_free_rate.rename(columns={'Risk-Free Return Rate (One Month Treasury Bill Rate)': 'daily_risk_free_rate'}, inplace=True)

# Handle unmatched date format
def parse_dates(date_series):
    return pd.to_datetime(date_series, dayfirst=False)

daily_risk_free_rate['date'] = parse_dates(daily_risk_free_rate['date'])

for stock, stock_data in training_stock_df.items():
    training_stock_df[stock] = pd.merge(stock_data, daily_risk_free_rate, on='date', how='inner')
    for halflife in [5, 10, 21]:
        downside_deviation = training_feature_df[stock][['date', f'{stock}_downside_deviation_{halflife}']]
        training_stock_df[stock] = pd.merge(training_stock_df[stock], downside_deviation, on='date', how='inner')

for stock, stock_data in testing_stock_df.items():
    testing_stock_df[stock] = pd.merge(stock_data, daily_risk_free_rate, on='date', how='inner')
    for halflife in [5, 10, 21]:
        downside_deviation = testing_feature_df[stock][['date', f'{stock}_downside_deviation_{halflife}']]
        testing_stock_df[stock] = pd.merge(testing_stock_df[stock], downside_deviation, on='date', how='inner')

In [ ]:
# Calculating the Sortino ratio
def sortino_ratio(daily_return, risk_free_rate, downside_deviation):
    return (daily_return - risk_free_rate) / downside_deviation

# Training
for stock, stock_data in training_stock_df.items():
    for halflife in [5, 10, 21]:
        daily_return = stock_data[f'{stock}_daily_return']
        risk_free_rate = stock_data['daily_risk_free_rate']
        downside_deviation = stock_data[f'{stock}_downside_deviation_{halflife}']
        training_feature_df[stock][f'{stock}_sortino_ratio_{halflife}'] = sortino_ratio(daily_return, risk_free_rate, downside_deviation)

        if check_weird_data(training_feature_df[stock][f'{stock}_sortino_ratio_{halflife}']):
            print(f"{stock} sortino ratio {halflife} contains weird data in training data")
            training_weird_columns_dict[stock].append(f'{stock}_sortino_ratio_{halflife}')

# Testing
for stock, stock_data in testing_stock_df.items():
    for halflife in [5, 10, 21]:
        daily_return = stock_data[f'{stock}_daily_return']
        risk_free_rate = stock_data['daily_risk_free_rate']
        downside_deviation = stock_data[f'{stock}_downside_deviation_{halflife}']
        testing_feature_df[stock][f'{stock}_sortino_ratio_{halflife}'] = sortino_ratio(daily_return, risk_free_rate, downside_deviation)

        if check_weird_data(testing_feature_df[stock][f'{stock}_sortino_ratio_{halflife}']):
            print(f"{stock} sortino ratio {halflife} contains weird data in testing data")
            testing_weird_columns_dict[stock].append(f'{stock}_sortino_ratio_{halflife}')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

In [ ]:
print(testing_stock_df['AAPL'].head().to_string())
print(training_stock_df['AAPL'].head().to_string())

#### Feature 10: 12 Month Momentum (mom12m) (Self-Constructed)

In [ ]:
def momentum_12m(stock_price):
    price_1m_ago = stock_price.shift(21)
    price_12m_ago = stock_price.shift(252)
    return price_1m_ago / price_12m_ago - 1

# Training
for stock, stock_data in training_stock_df.items():
    training_feature_df[stock][f'{stock}_mom12m'] = momentum_12m(stock_data[stock])
    if check_weird_data(training_feature_df[stock][f'{stock}_mom12m']):
        print(f"Feature {stock}_mom12m contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_mom12m')

# Notice that the testing period is too short to calculate the 12-month momentum
# So it will be calculated in later stage if this feature is selected

In [ ]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

# Clear weird_columns_dict
for stock in stock_list:
    training_weird_columns_dict[stock] = []

#### Feature 11: short-term reversal (mom1m) (Self-Constructed)

In [ ]:
def momentum_1m(stock_price):
    stock_price_1m_ago = stock_price.shift(21)
    return stock_price / stock_price_1m_ago - 1

# Training
for stock, stock_data in training_stock_df.items():
    training_feature_df[stock][f'{stock}_mom1m'] = momentum_1m(stock_data[stock])
    if check_weird_data(training_feature_df[stock][f'{stock}_mom1m']):
        print(f"Feature {stock}_mom1m contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_mom1m')

# Testing
for stock, stock_data in testing_stock_df.items():
    testing_feature_df[stock][f'{stock}_mom1m'] = momentum_1m(stock_data[stock])
    if check_weird_data(testing_feature_df[stock][f'{stock}_mom1m']):
        print(f"Feature {stock}_mom1m contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_mom1m')

In [ ]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")

In [ ]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

for stock in stock_list:
    testing_weird_columns_dict[stock] = []

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 12: 6-month momentum (mom6m) (Self-Constructed)

In [ ]:
def momentum_6m(stock_price):
    stock_price_6m_ago = stock_price.shift(126)
    return stock_price / stock_price_6m_ago - 1

# Training
for stock, stock_data in training_stock_df.items():
    training_feature_df[stock][f'{stock}_mom6m'] = momentum_6m(stock_data[stock])
    if check_weird_data(training_feature_df[stock][f'{stock}_mom6m']):
        print(f"Feature {stock}_mom6m contains weird data")
        training_weird_columns_dict[stock].append(f'{stock}_mom6m')

# Testing
for stock, stock_data in testing_stock_df.items():
    testing_feature_df[stock][f'{stock}_mom6m'] = momentum_6m(stock_data[stock])
    if check_weird_data(testing_feature_df[stock][f'{stock}_mom6m']):
        print(f"Feature {stock}_mom6m contains weird data")
        testing_weird_columns_dict[stock].append(f'{stock}_mom6m')

In [ ]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")

In [ ]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

for stock in stock_list:
    testing_weird_columns_dict[stock] = []

# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 14: Log Market Equity (mvel1)

In [ ]:
'''
Notice that we already have the daily price data,
and I found shareoutstanding data can be found on wrds, shrout is the number of shares outstanding
https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-events/share-outstanding/
PERMNO is the unique identifier for each stock
AMZN: 84788
GOOGL: 90319
MSFT: 10107
NVDA: 86580
AAPL: 14593
NFLX: 89393
AVGO: 93002
TSLA: 93436
META: 13407
'''
permno_dict = config_data['permno_dict']
def share_outstanding_preprocessing():
    # Load the share outstanding data
    share_outstanding_data = pd.read_excel(f"{DATA_DIR}/company_fundamentals/{stock}_share_outstanding_raw.xlsx")

    # Make sure the downloaded data is correct
    if int(permno_dict[stock]) != int(share_outstanding_data['PERMNO'][0]):
        print("You downloaded the wrong data for {stock}")
        exit()

    # rename the column from Shares Observation Date to date
    share_outstanding_data = share_outstanding_data.rename(columns={'Shares Observation Date': 'date'})
    # Convert the date column to datetime
    share_outstanding_data['date'] =  parse_dates(share_outstanding_data['date'])
    # Drop PERMNO column
    share_outstanding_data = share_outstanding_data.drop(columns=['PERMNO'])
    # Notice that the original data is expressed in thousand, so need to multiple the data by 1000
    share_outstanding_data['Shares Outstanding'] *= 1000

    return share_outstanding_data

def mvel1(stock_price, share_outstanding):
    return np.log(stock_price * share_outstanding + 1e-9)

# Share outstanding data (Training)
for stock, stock_data in training_stock_df.items():
    share_outstanding_data = share_outstanding_preprocessing()
    # Merge the share outstanding data with the stock price data (Training)
    training_date = parse_dates(training_stock_df[stock]['date'])
    training_share_outstanding = pd.merge(training_date, share_outstanding_data, on='date', how='outer')

    # Backfill the share outstanding data
    training_share_outstanding = training_share_outstanding.bfill()

    training_feature_df[stock][f'{stock}_mvel1'] = mvel1(stock_data[stock], training_share_outstanding['Shares Outstanding'])

    if check_weird_data(training_feature_df[stock][f'{stock}_mvel1']):
        print(f"Feature {stock}_mvel1 contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_mvel1')

# Share outstanding data (Testing)
for stock, stock_data in testing_stock_df.items():
    # Merge the share outstanding data with the stock price data (Testing)
    testing_date = parse_dates(testing_stock_df[stock]['date'])
    testing_share_outstanding = pd.merge(testing_date, share_outstanding_data, on='date', how='outer')

    # Backfill the share outstanding data
    testing_share_outstanding = testing_share_outstanding.bfill()

    # There are still some missing data in the share outstanding data
    # So we need to forward fill the data
    testing_share_outstanding = testing_share_outstanding.ffill()

    testing_feature_df[stock][f'{stock}_mvel1'] = mvel1(stock_data[stock], testing_share_outstanding['Shares Outstanding'])

    if check_weird_data(testing_feature_df[stock][f'{stock}_mvel1']):
        print(f"Feature {stock}_mvel1 contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_mvel1')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 15: Standard deviation of daily returns from month t-1 (retvol)

In [ ]:
"""
Data Source: https://wrds-www.wharton.upenn.edu/pages/get-data/contributed-data-forms/global-factor-data/
PERMNO is the unique identifier for each stock
Amazon: 84788
Google: 90319
Microsoft: 10107
Nvidia: 86580
Apple: 14593
NFLX: 89393
AVGO: 93002
Tesla: 93436
Meta: 13407
"""
def get_rvol_21d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_col = ['Day of last price observation (date)', 'Return volatility (rvol_21d)']
    rvol_21d = global_factor[target_col].copy()  # Use .copy() to avoid SettingWithCopyWarning

    # Rename the columns
    rvol_21d.rename(columns={'Return volatility (rvol_21d)': f'{stock}_rvol_21d'}, inplace=True)

    # Parse the date column
    rvol_21d['Day of last price observation (date)'] = parse_dates(rvol_21d['Day of last price observation (date)'])

    return rvol_21d

# Training
for stock, stock_data in training_stock_df.items():
    rvol_21d = get_rvol_21d(stock)

    # Merge the rvol_21d data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], rvol_21d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values 
    training_feature_df[stock][f'{stock}_rvol_21d'] = training_feature_df[stock][f'{stock}_rvol_21d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_rvol_21d']):
        print(f"Feature f'{stock}_rvol_21d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_rvol_21d')

# Testing
for stock, stock_data in testing_stock_df.items():
    rvol_21d = get_rvol_21d(stock)

    # Merge the rvol_21d data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], rvol_21d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_rvol_21d'] = testing_feature_df[stock][f'{stock}_rvol_21d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_rvol_21d']):
        print(f"Feature f'{stock}_rvol_21d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_rvol_21d')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 16-18: Change in Shares - 1 Month (chcsho_1m, chcsho_3m, chcsho_6m)

In [ ]:
def get_chcsho(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Change in Shares - 1 Month (chcsho_1m)', 'Change in Shares - 3 Month (chcsho_3m)', 'Change in Shares - 6 Month (chcsho_6m)']
    chcsho = global_factor[target_cols].copy()  # Use .copy() to avoid SettingWithCopyWarning

    # Rename the columns
    chcsho.rename(columns={'Change in Shares - 1 Month (chcsho_1m)': f'{stock}_chcsho_1m', 'Change in Shares - 3 Month (chcsho_3m)': f'{stock}_chcsho_3m', 'Change in Shares - 6 Month (chcsho_6m)': f'{stock}_chcsho_6m'}, inplace=True)

    # Parse the date column
    chcsho['Day of last price observation (date)'] = parse_dates(chcsho['Day of last price observation (date)'])

    return chcsho


# Training
for stock, stock_data in training_stock_df.items():
    chcsho = get_chcsho(stock)

    # Merge the chcsho data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], chcsho, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_chcsho_1m'] = training_feature_df[stock][f'{stock}_chcsho_1m'].bfill()
    training_feature_df[stock][f'{stock}_chcsho_3m'] = training_feature_df[stock][f'{stock}_chcsho_3m'].bfill()
    training_feature_df[stock][f'{stock}_chcsho_6m'] = training_feature_df[stock][f'{stock}_chcsho_6m'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_chcsho_1m']):
        print(f"Feature f'{stock}_chcsho_1m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_chcsho_1m')

    if check_weird_data(training_feature_df[stock][f'{stock}_chcsho_3m']):
        print(f"Feature f'{stock}_chcsho_3m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_chcsho_3m')

    if check_weird_data(training_feature_df[stock][f'{stock}_chcsho_6m']):
        print(f"Feature f'{stock}_chcsho_6m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_chcsho_6m')


# Testing
for stock, stock_data in testing_stock_df.items():
    chcsho = get_chcsho(stock)

    # Merge the chcsho data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], chcsho, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_chcsho_1m'] = testing_feature_df[stock][f'{stock}_chcsho_1m'].bfill()
    testing_feature_df[stock][f'{stock}_chcsho_3m'] = testing_feature_df[stock][f'{stock}_chcsho_3m'].bfill()
    testing_feature_df[stock][f'{stock}_chcsho_6m'] = testing_feature_df[stock][f'{stock}_chcsho_6m'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_chcsho_1m']):
        print(f"Feature f'{stock}_chcsho_1m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_chcsho_1m')

    if check_weird_data(testing_feature_df[stock][f'{stock}_chcsho_3m']):
        print(f"Feature f'{stock}_chcsho_3m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_chcsho_3m')

    if check_weird_data(testing_feature_df[stock][f'{stock}_chcsho_6m']):
        print(f"Feature f'{stock}_chcsho_6m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_chcsho_6m')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 19: Amihud illiquidity (ami_126d)

In [ ]:
def get_ami_126d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Amihud Measure (ami_126d)']
    ami_126d = global_factor[target_cols].copy()
    

    # Rename the columns
    ami_126d.rename(columns={'Amihud Measure (ami_126d)': f'{stock}_ami_126d'}, inplace=True)

    # Parse the date column
    ami_126d['Day of last price observation (date)'] = parse_dates(ami_126d['Day of last price observation (date)'])

    return ami_126d

# Training
for stock, stock_data in training_stock_df.items():
    ami_126d = get_ami_126d(stock)

    # Merge the ami_126d data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], ami_126d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_ami_126d'] = training_feature_df[stock][f'{stock}_ami_126d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_ami_126d']):
        print(f"Feature f'{stock}_ami_126d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_ami_126d')

# Testing
for stock, stock_data in testing_stock_df.items():
    ami_126d = get_ami_126d(stock)

    # Merge the ami_126d data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], ami_126d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_ami_126d'] = testing_feature_df[stock][f'{stock}_ami_126d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_ami_126d']):
        print(f"Feature f'{stock}_ami_126d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_ami_126d')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 20: Age of the firms in months (age)

In [ ]:
def get_age(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Firm age (age)']
    age = global_factor[target_cols].copy()
    

    # Rename the columns
    age.rename(columns={'Firm age (age)': f'{stock}_age'}, inplace=True)

    # Parse the date column
    age['Day of last price observation (date)'] = parse_dates(age['Day of last price observation (date)'])

    return age

# Training
for stock, stock_data in training_stock_df.items():
    age = get_age(stock)

    # Merge the age data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], age, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_age'] = training_feature_df[stock][f'{stock}_age'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_age']):
        print(f"Feature f'{stock}_age' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_age')

# Testing
for stock, stock_data in testing_stock_df.items():
    age = get_age(stock)

    # Merge the age data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], age, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_age'] = testing_feature_df[stock][f'{stock}_age'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_age']):
        print(f"Feature f'{stock}_age' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_age')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 21: Change in 6-month momentum (chmom) (Self-Constructed)

In [ ]:
def get_chmom(stock_price):
    price_1m_ago = stock_price.shift(21)
    price_6m_ago = stock_price.shift(126)

    price_7m_ago = stock_price.shift(147)
    price_12m_ago = stock_price.shift(252)
    return (price_1m_ago / price_6m_ago) - 1 - ( (price_7m_ago / price_12m_ago) - 1 )

# Training
for stock, stock_data in training_stock_df.items():
    training_feature_df[stock][f'{stock}_chmom'] = get_chmom(stock_data[stock])
    if check_weird_data(training_feature_df[stock][f'{stock}_chmom']):
        print(f"Feature {stock}_chmom contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_chmom')

# Similarly, the testing period is too short to calculate the change in 6-month momentum
# So it will be calculated in later stage if this feature is selected

In [ ]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

In [ ]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 22: Maximum daily return (maxret) (Self-Constructed)

In [ ]:
def get_maxret(stock_df, stock_ticket):
    # Create a copy to avoid SettingWithCopy issues
    df = stock_df.copy()
    df.loc[:, 'year_month'] = df['date'].dt.to_period('M')
    monthly_max = df.groupby('year_month')[f'{stock_ticket}_daily_return'].max().reset_index()
    # Rename the computed column
    monthly_max = monthly_max.rename(columns={f'{stock_ticket}_daily_return': f'{stock_ticket}_maxret'})
    # Shift the max returns down by one period so that each row corresponds to calendar month t-1
    monthly_max[f'{stock_ticket}_maxret'] = monthly_max[f'{stock_ticket}_maxret'].shift(1)
    # Convert the period back to a timestamp; here we choose the month-end timestamp
    monthly_max['year_month'] = monthly_max['year_month'].dt.to_timestamp('M')
    return monthly_max

# Training
for stock, stock_data in training_stock_df.items():
    maxret = get_maxret(stock_data, stock)
    training_feature_df[stock] = pd.merge(training_feature_df[stock], maxret, how='left', left_on='date', right_on='year_month')
    training_feature_df[stock].drop(columns=['year_month'], inplace=True)
    training_feature_df[stock][f'{stock}_maxret'] = training_feature_df[stock][f'{stock}_maxret'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_maxret']):
        print(f"Feature {stock}_maxret contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_maxret')

# Testing
for stock, stock_data in testing_stock_df.items():
    maxret = get_maxret(stock_data, stock)
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], maxret, how='left', left_on='date', right_on='year_month')
    testing_feature_df[stock].drop(columns=['year_month'], inplace=True)
    testing_feature_df[stock][f'{stock}_maxret'] = testing_feature_df[stock][f'{stock}_maxret'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_maxret']):
        print(f"Feature {stock}_maxret contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_maxret')

In [ ]:
# Training
for stock, weird_columns in training_weird_columns_dict.items():
    if weird_columns:
        training_feature_df[stock] = handle_weird_data_with_knn_imputer(training_feature_df[stock], weird_columns)

# Testing
for stock, weird_columns in testing_weird_columns_dict.items():
    if weird_columns:
        testing_feature_df[stock] = handle_weird_data_with_knn_imputer(testing_feature_df[stock], weird_columns)

# Check the data again
for stock, stock_data in training_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in training data")

for stock, stock_data in testing_feature_df.items():
    if check_weird_data(stock_data):
        print(f"The {stock} stock price data still contains weird data in testing data")


In [ ]:
# Clear weird_columns_dict, only clear the value, not the key
for stock in stock_list:
    training_weird_columns_dict[stock] = []

for stock in stock_list:
    testing_weird_columns_dict[stock] = []

# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())


#### Feature 23: Dollar trading volume (dolvol)

In [ ]:
def get_dolvol(stock):
    gobal_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Dollar trading volume (dolvol)']
    dolvol = gobal_factor[target_cols].copy()
    
    # Rename the columns
    dolvol.rename(columns={'Dollar trading volume (dolvol)': f'{stock}_dolvol'}, inplace=True)

    # Parse the date column
    dolvol['Day of last price observation (date)'] = parse_dates(dolvol['Day of last price observation (date)'])

    return dolvol

# Training
for stock, stock_data in training_stock_df.items():
    dolvol = get_dolvol(stock)

    # Merge the dolvol data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], dolvol, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_dolvol'] = training_feature_df[stock][f'{stock}_dolvol'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_dolvol']):
        print(f"Feature f'{stock}_dolvol' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_dolvol')

# Testing
for stock, stock_data in testing_stock_df.items():
    dolvol = get_dolvol(stock)

    # Merge the dolvol data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], dolvol, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_dolvol'] = testing_feature_df[stock][f'{stock}_dolvol'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_dolvol']):
        print(f"Feature f'{stock}_dolvol' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_dolvol')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 24: 60 Month CAPM Beta (beta_60m)

In [ ]:
def get_beta_60m(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Market Beta (beta_60m)']
    beta_60m = global_factor[target_cols].copy()
    

    # Rename the columns
    beta_60m.rename(columns={'Market Beta (beta_60m)': f'{stock}_beta_60m'}, inplace=True)

    # Parse the date column
    beta_60m['Day of last price observation (date)'] = parse_dates(beta_60m['Day of last price observation (date)'])

    return beta_60m

# Training
for stock, stock_data in training_stock_df.items():
    beta_60m = get_beta_60m(stock)

    # Merge the beta_60m data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], beta_60m, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_beta_60m'] = training_feature_df[stock][f'{stock}_beta_60m'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_beta_60m']):
        print(f"Feature f'{stock}_beta_60m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_beta_60m')

# Testing
for stock, stock_data in testing_stock_df.items():
    beta_60m = get_beta_60m(stock)

    # Merge the beta_60m data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], beta_60m, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_beta_60m'] = testing_feature_df[stock][f'{stock}_beta_60m'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_beta_60m']):
        print(f"Feature f'{stock}_beta_60m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_beta_60m')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 25-27: Number of zero trades with turnover as tiebreaker (1 month, 6 months, 12 months) (zero_trades_21d, zero_trades_126d, zero_trades_252d)

In [ ]:
def get_zero_trades(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Number of zero trades with turnover as tiebreaker (1 month) (ze', 'Number of zero trades with turnover as tiebreaker (6 months) (z', 'Number of zero trades with turnover as tiebreaker (12 months) (' ]
    zero_trades = global_factor[target_cols].copy()
    

    # Rename the columns
    zero_trades.rename(columns={
        'Number of zero trades with turnover as tiebreaker (1 month) (ze': f'{stock}_zero_trades_1m', 
        'Number of zero trades with turnover as tiebreaker (6 months) (z': f'{stock}_zero_trades_6m', 
        'Number of zero trades with turnover as tiebreaker (12 months) (': f'{stock}_zero_trades_12m'}, inplace=True)

    # Parse the date column
    zero_trades['Day of last price observation (date)'] = parse_dates(zero_trades['Day of last price observation (date)'])

    return zero_trades

# Training
for stock, stock_data in training_stock_df.items():
    zero_trades = get_zero_trades(stock)

    # Merge the zero_trades data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], zero_trades, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_zero_trades_1m'] = training_feature_df[stock][f'{stock}_zero_trades_1m'].bfill()
    training_feature_df[stock][f'{stock}_zero_trades_6m'] = training_feature_df[stock][f'{stock}_zero_trades_6m'].bfill()
    training_feature_df[stock][f'{stock}_zero_trades_12m'] = training_feature_df[stock][f'{stock}_zero_trades_12m'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_zero_trades_1m']):
        print(f"Feature f'{stock}_zero_trades_1m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_zero_trades_1m')

    if check_weird_data(training_feature_df[stock][f'{stock}_zero_trades_6m']):
        print(f"Feature f'{stock}_zero_trades_6m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_zero_trades_6m')

    if check_weird_data(training_feature_df[stock][f'{stock}_zero_trades_12m']):
        print(f"Feature f'{stock}_zero_trades_12m' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_zero_trades_12m')

# Testing
for stock, stock_data in testing_stock_df.items():
    zero_trades = get_zero_trades(stock)

    # Merge the zero_trades data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], zero_trades, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_zero_trades_1m'] = testing_feature_df[stock][f'{stock}_zero_trades_1m'].bfill()
    testing_feature_df[stock][f'{stock}_zero_trades_6m'] = testing_feature_df[stock][f'{stock}_zero_trades_6m'].bfill()
    testing_feature_df[stock][f'{stock}_zero_trades_12m'] = testing_feature_df[stock][f'{stock}_zero_trades_12m'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_zero_trades_1m']):
        print(f"Feature f'{stock}_zero_trades_1m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_zero_trades_1m')

    if check_weird_data(testing_feature_df[stock][f'{stock}_zero_trades_6m']):
        print(f"Feature f'{stock}_zero_trades_6m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_zero_trades_6m')

    if check_weird_data(testing_feature_df[stock][f'{stock}_zero_trades_12m']):
        print(f"Feature f'{stock}_zero_trades_12m' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_zero_trades_12m')


In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 13: Frazzini-Pedersen market beta (betabab_1260d)

In [ ]:
def get_betabab_1260d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Frazzini-Pedersen market beta (betabab_1260d)']
    betabab_1260d = global_factor[target_cols].copy()
    

    # Rename the columns
    betabab_1260d.rename(columns={'Frazzini-Pedersen market beta (betabab_1260d)': f'{stock}_betabab_1260d'}, inplace=True)

    # Parse the date column
    betabab_1260d['Day of last price observation (date)'] = parse_dates(betabab_1260d['Day of last price observation (date)'])

    return betabab_1260d

# Training
for stock, stock_data in training_stock_df.items():
    betabab_1260d = get_betabab_1260d(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], betabab_1260d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_betabab_1260d'] = training_feature_df[stock][f'{stock}_betabab_1260d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_betabab_1260d']):
        print(f"Feature f'{stock}_betabab_1260d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_betabab_1260d')

# Testing
for stock, stock_data in testing_stock_df.items():
    betabab_1260d = get_betabab_1260d(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], betabab_1260d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_betabab_1260d'] = testing_feature_df[stock][f'{stock}_betabab_1260d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_betabab_1260d']):
        print(f"Feature f'{stock}_betabab_1260d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_betabab_1260d')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 26: Earnings variability (earnings_variability)

In [ ]:
def get_earnings_variability(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Earnings variability (earnings_variability)']
    earnings_variability = global_factor[target_cols].copy()

    # Rename the columns
    earnings_variability.rename(columns={'Earnings variability (earnings_variability)': f'{stock}_earnings_variability'}, inplace=True)

    # Parse the date column
    earnings_variability['Day of last price observation (date)'] = parse_dates(earnings_variability['Day of last price observation (date)'])

    return earnings_variability

# Training
for stock, stock_data in training_stock_df.items():
    earnings_variability = get_earnings_variability(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], earnings_variability, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_earnings_variability'] = training_feature_df[stock][f'{stock}_earnings_variability'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_earnings_variability']):
        print(f"Feature f'{stock}_earnings_variability' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_earnings_variability')

# Testing
for stock, stock_data in testing_stock_df.items():
    earnings_variability = get_earnings_variability(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], earnings_variability, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_earnings_variability'] = testing_feature_df[stock][f'{stock}_earnings_variability'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_earnings_variability']):
        print(f"Feature f'{stock}_earnings_variability' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_earnings_variability')


In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 27: Operating Cash Flow to Sales Quarterly Volatility (ocfq_saleq_std)

In [ ]:
def get_ocfq_saleq_std(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Cash flow volatility (ocfq_saleq_std)']
    ocfq_saleq_std = global_factor[target_cols].copy()

    # Rename the columns
    ocfq_saleq_std.rename(columns={'Cash flow volatility (ocfq_saleq_std)': f'{stock}_ocfq_saleq_std'}, inplace=True)

    # Parse the date column
    ocfq_saleq_std['Day of last price observation (date)'] = parse_dates(ocfq_saleq_std['Day of last price observation (date)'])

    return ocfq_saleq_std

# Training
for stock, stock_data in training_stock_df.items():
    ocfq_saleq_std = get_ocfq_saleq_std(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], ocfq_saleq_std, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_ocfq_saleq_std'] = training_feature_df[stock][f'{stock}_ocfq_saleq_std'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_ocfq_saleq_std']):
        print(f"Feature f'{stock}_ocfq_saleq_std' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_ocfq_saleq_std')

# Testing
for stock, stock_data in testing_stock_df.items():
    ocfq_saleq_std = get_ocfq_saleq_std(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], ocfq_saleq_std, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_ocfq_saleq_std'] = testing_feature_df[stock][f'{stock}_ocfq_saleq_std'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_ocfq_saleq_std']):
        print(f"Feature f'{stock}_ocfq_saleq_std' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_ocfq_saleq_std')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 28: Turnover (turnover_126d)

In [ ]:
def get_turnover_126d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Share turnover (turnover_126d)']
    turnover_126d = global_factor[target_cols].copy()

    # Rename the columns
    turnover_126d.rename(columns={'Share turnover (turnover_126d)': f'{stock}_turnover_126d'}, inplace=True)

    # Parse the date column
    turnover_126d['Day of last price observation (date)'] = parse_dates(turnover_126d['Day of last price observation (date)'])

    return turnover_126d

# Training
for stock, stock_data in training_stock_df.items():
    turnover_126d = get_turnover_126d(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], turnover_126d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_turnover_126d'] = training_feature_df[stock][f'{stock}_turnover_126d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_turnover_126d']):
        print(f"Feature f'{stock}_turnover_126d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_turnover_126d')

# Testing
for stock, stock_data in testing_stock_df.items():
    turnover_126d = get_turnover_126d(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], turnover_126d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_turnover_126d'] = testing_feature_df[stock][f'{stock}_turnover_126d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_turnover_126d']):
        print(f"Feature f'{stock}_turnover_126d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_turnover_126d')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 29: Current price to high price over last year (prc_highprc_252d)

In [ ]:
def get_prc_highprc_252d(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Current price to high price over last year (prc_highprc_252d)']
    prc_highprc_252d = global_factor[target_cols].copy()

    # Rename the columns
    prc_highprc_252d.rename(columns={target_cols[1]: f'{stock}_prc_highprc_252d'}, inplace=True)

    # Parse the date column
    prc_highprc_252d[target_cols[0]] = parse_dates(prc_highprc_252d[target_cols[0]])

    return prc_highprc_252d

# Training
for stock, stock_data in training_stock_df.items():
    prc_highprc_252d = get_prc_highprc_252d(stock)

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], prc_highprc_252d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}_prc_highprc_252d'] = training_feature_df[stock][f'{stock}_prc_highprc_252d'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}_prc_highprc_252d']):
        print(f"Feature f'{stock}_prc_highprc_252d' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}_prc_highprc_252d')

# Testing
for stock, stock_data in testing_stock_df.items():
    prc_highprc_252d = get_prc_highprc_252d(stock)

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], prc_highprc_252d, how='left', left_on='date', right_on='Day of last price observation (date)')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=['Day of last price observation (date)'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}_prc_highprc_252d'] = testing_feature_df[stock][f'{stock}_prc_highprc_252d'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}_prc_highprc_252d']):
        print(f"Feature f'{stock}_prc_highprc_252d' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}_prc_highprc_252d')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 30: Residual momentum t-6 to t-1 (resff3_6_1)

In [ ]:
def get_resff3_6_1(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Residual momentum t-6 to t-1 (resff3_6_1)']
    new_cols = global_factor[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_resff3_6_1'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_resff3_6_1(stock)
    substring = '_resff3_6_1'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_resff3_6_1(stock)
    substring = '_resff3_6_1'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')


In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 31: 1 Year Non-Annual Seasonality (seas_1_1na)

In [ ]:
def get_seas_1_1na(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Year 1-lagged return, annual (seas_1_1an)']
    new_cols = global_factor[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_seas_1_1na'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_seas_1_1na(stock)
    substring = '_seas_1_1na'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_seas_1_1na(stock)
    substring = '_seas_1_1na'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')


In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 32: Capital turnover (at_turnover)

In [ ]:
def get_at_turnover(stock):
    global_factor = pd.read_excel(f"{DATA_DIR}/global_factor/{stock}_global_factor.xlsx")
    target_cols = ['Day of last price observation (date)', 'Capital turnover (at_turnover)']
    new_cols = global_factor[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'{stock}_at_turnover'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_at_turnover(stock)
    substring = '_at_turnover'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{stock}{substring}'] = training_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{stock}{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_at_turnover(stock)
    substring = '_at_turnover'
    date_substring = 'Day of last price observation (date)'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{stock}{substring}'] = testing_feature_df[stock][f'{stock}{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{stock}{substring}']):
        print(f"Feature f'{stock}{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{stock}{substring}')

In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())

#### Feature 33: US 30 Day Bill Returns (T-30)

In [ ]:
def get_T_30():
    global_factor = pd.read_excel(f"{DATA_DIR}/Monetary and Market Condition/US 30 Day Bill Returns.xlsx")
    target_cols = ['Calendar Date', '30 Day Bill Returns']
    new_cols = global_factor[target_cols].copy()

    # Rename the columns
    new_cols.rename(columns={target_cols[1]: f'T_30'}, inplace=True)

    # Parse the date column
    new_cols[target_cols[0]] = parse_dates(new_cols[target_cols[0]])

    return new_cols

# Training
for stock, stock_data in training_stock_df.items():
    new_cols = get_T_30()
    substring = 'T_30'
    date_substring = 'Calendar Date'

    # Merge the data with the stock data
    training_feature_df[stock] = pd.merge(training_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    training_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    training_feature_df[stock][f'{substring}'] = training_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(training_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in training data")
        training_weird_columns_dict[stock].append(f'{substring}')

# Testing
for stock, stock_data in testing_stock_df.items():
    new_cols = get_T_30()
    substring = 'T_30'
    date_substring = 'Calendar Date'

    # Merge the data with the stock data
    testing_feature_df[stock] = pd.merge(testing_feature_df[stock], new_cols, how='left', left_on='date', right_on=f'{date_substring}')
    # Drop the 'Day of last price observation (date)' column
    testing_feature_df[stock].drop(columns=[f'{date_substring}'], inplace=True)

    # Backfill the missing values
    testing_feature_df[stock][f'{substring}'] = testing_feature_df[stock][f'{substring}'].bfill()

    if check_weird_data(testing_feature_df[stock][f'{substring}']):
        print(f"Feature f'{substring}' contains weird data in testing data")
        testing_weird_columns_dict[stock].append(f'{substring}')


In [ ]:
# Print the result to ensure the data is correct
print(training_feature_df['AAPL'].head().to_string())
print(testing_feature_df['AAPL'].head().to_string())